In [1]:
import numpy as np
import pandas as pd
import customFunctions as fct
import foxes
import foxes.variables as FV
import foxes.input.farm_layout as layout
from foxes import algorithms
from foxes.output import FarmResultsEval
import os
os.environ["FOXES_ENGINE"]    = "threads"
os.environ["FOXES_N_PROCS"]   = "1"

# 1) Load & prepare ERA5 from reanalysis.csv
era5 = pd.read_csv("reanalysis.csv", index_col=0, parse_dates=True)
era5.rename(columns={"WS100":"WS","WD100":"WD"}, inplace=True)
era5.replace(-999, np.nan, inplace=True)
era5 = era5.resample("h").mean()
era5.index = pd.to_datetime(era5.index)
era5 = era5.loc["2006-01-01":"2006-12-31"]

# 2) Ensure turbulence intensity column exists
if "TI" not in era5.columns:
    era5["TI"] = 0.10  

# 3) Build FOXES TimeSeries states using the cleaned WS/WD
states = foxes.input.states.Timeseries(
    data_source=era5,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "WS", FV.WD: "WD", FV.TI: "TI"},
    fixed_vars={FV.RHO: 1.225},
)

# 4) Create the wind farm layout for N-10.1
farm = foxes.WindFarm(name="N-10.1")

layout_data = pd.read_csv(
    "turbine-info/coordinates/area_of_interest/layout-N-10.1.geom.csv",
    index_col=None
)
# add turbines using NREL5MW model
layout.add_from_csv(
    farm,
    layout_data,
    turbine_models=["NREL5MW"],
    verbosity=0
)

# 5) Set up & run FOXES (internal wake only)
algo = algorithms.Downwind(
    farm,
    states,
    rotor_model="centre",
    wake_models=["Bastankhah2014_quadratic_ka02"],
    verbosity=0
)

results = algo.calc_farm(
    calc_parameters={"chunk_size_states": 1000}
)

# 6) Aggregate results & compute yield/efficiency
eval_ = FarmResultsEval(results)
eval_.add_capacity(algo)               
eval_.add_capacity(algo, ambient=True) 
eval_.add_efficiency()                 

# Compute per turbine annual yields
yld_net_da = eval_.calc_turbine_yield(algo, annual=True)
yld_amb_da = eval_.calc_turbine_yield(algo, annual=True, ambient=True)

yld_net = yld_net_da.values.squeeze()
yld_amb = yld_amb_da.values.squeeze()

# Compute per turbine efficiency
eff_per = yld_net / yld_amb

# Build a pandas DataFrame with the turbine names as index
turbine_stats = pd.DataFrame({
    "Ambient Yield [GWh]": yld_amb,
    "Net Yield     [GWh]": yld_net,
    "Efficiency         ": eff_per
}, index=algo.farm.turbine_names)

# 7) Farm summary & print
farm_ambient = eval_.calc_mean_farm_power(ambient=True) / 1000  
farm_net     = eval_.calc_mean_farm_power()          / 1000     
farm_eff     = eval_.calc_farm_efficiency()                   
annual_yld   = yld_net.sum()                                  

print(f"Farm ambient power: {farm_ambient:.1f} MW")
print(f"Farm net power    : {farm_net:.1f} MW")
print(f"Farm efficiency   : {farm_eff:.2f}")
print(f"Annual yield      : {annual_yld:.2f} GWh\n")

print("Per-turbine results:")
print(turbine_stats)


Selecting 'DefaultEngine(n_procs=16, chunk_size_states=None, chunk_size_points=None)'
DefaultEngine: Selecting engine 'process'
ProcessEngine: Calculating 8760 states for 133 turbines
ProcessEngine: Computing 16 chunks using 16 processes


100%|██████████| 16/16 [00:38<00:00,  2.40s/it]


Capacity added to farm results
Ambient capacity added to farm results
Efficiency added to farm results
Farm ambient power: 375.9 MW
Farm net power    : 355.0 MW
Farm efficiency   : 0.94
Annual yield      : 3109.90 GWh

Per-turbine results:
      Ambient Yield [GWh]  Net Yield     [GWh]  Efficiency         
T0              24.755557            24.101155             0.973565
T1              24.755557            24.028297             0.970622
T2              24.755557            24.024398             0.970465
T3              24.755557            24.261753             0.980053
T4              24.755557            23.714143             0.957932
...                   ...                  ...                  ...
T128            24.755557            23.274699             0.940181
T129            24.755557            23.074191             0.932081
T130            24.755557            23.190648             0.936786
T131            24.755557            23.117964             0.933849
T132        